Data pipeline adalah serangkaian proses (filter, shuffle, map, dll) yang dilakukan pada data secara berantai, menggunakan tensorflow data pipeline, salah satu keuntungannya adalah kita bisa melakukan caching dan prefetching pada data yang membuat pemrosesan data menjadi cepat.

In [1]:
import tensorflow as tf

import numpy as np

#### Dataset random

Misal datasetnya merupakan data random yang digenerate seperti di bawah ini

In [2]:
random_arr= np.random.randint(-100,500, 10)
random_arr

array([407, 310, 239, 249, 334, 231, -73,  57, 364, 444])

Hal pertama yang harus kita lakukan untuk menerapkan tensorflow data pipeline adalah membuat data tersebut menjadi objek tensor

In [3]:
# Fungsi untuk mengubah list atau array menjadi objek tensor
tf_dataset= tf.data.Dataset.from_tensor_slices(random_arr)

tf_dataset

<TensorSliceDataset shapes: (), types: tf.int32>

Menampilkan data yang sekarang menjadi objek tensor

In [4]:
# Perulangan dari tf_dataset
for i in tf_dataset:
    # Jika ingin melakukan slice n elemen pada tf_dataset, lakukan tf_dataset.take(n)
    print(i)
    # Jika kita memprint i, yang akan tampil bukan nilanya langsung
    print(i.numpy())
    # Kita bisa memanggil fungsi numpy() untuk melakukan hal tersebut

tf.Tensor(407, shape=(), dtype=int32)
407
tf.Tensor(310, shape=(), dtype=int32)
310
tf.Tensor(239, shape=(), dtype=int32)
239
tf.Tensor(249, shape=(), dtype=int32)
249
tf.Tensor(334, shape=(), dtype=int32)
334
tf.Tensor(231, shape=(), dtype=int32)
231
tf.Tensor(-73, shape=(), dtype=int32)
-73
tf.Tensor(57, shape=(), dtype=int32)
57
tf.Tensor(364, shape=(), dtype=int32)
364
tf.Tensor(444, shape=(), dtype=int32)
444


Hal yang akan kita lakukan pada tf_dataset yaitu kumpulan dari beberapa proses, yaitu filter angka yang lebih besar dari 0, normalisasi dengan membagi 10, lalu shuffle (mengacak)

Filter

In [5]:
filter_ds= lambda x: x>0

tf_dataset= tf_dataset.filter(filter_ds)

In [6]:
for i in tf_dataset:
    print(i.numpy())

407
310
239
249
334
231
57
364
444


Normalisasi

In [7]:
tf_dataset= tf_dataset.map(lambda y: y/10)

for i in tf_dataset:
    print(i)

tf.Tensor(40.7, shape=(), dtype=float64)
tf.Tensor(31.0, shape=(), dtype=float64)
tf.Tensor(23.9, shape=(), dtype=float64)
tf.Tensor(24.9, shape=(), dtype=float64)
tf.Tensor(33.4, shape=(), dtype=float64)
tf.Tensor(23.1, shape=(), dtype=float64)
tf.Tensor(5.7, shape=(), dtype=float64)
tf.Tensor(36.4, shape=(), dtype=float64)
tf.Tensor(44.4, shape=(), dtype=float64)


Shuffle

In [8]:
tf_dataset= tf_dataset.shuffle(2)

for i in tf_dataset:
    print(i.numpy())

40.7
23.9
24.9
31.0
33.4
5.7
36.4
44.4
23.1


Semua proses diatas bisa dan disarankan dirangkum seperti ini

In [10]:
tf_dataset_2= tf.data.Dataset.from_tensor_slices(random_arr)

tf_dataset_2= tf_dataset_2.filter(filter_ds).map(lambda x: x/10).shuffle(2)

for i in tf_dataset_2:
    print(i.numpy())

31.0
40.7
23.9
33.4
23.1
24.9
5.7
44.4
36.4


**Caching**

Artinya men-cache data yang telah diproses, sehingga kita tidak perlu melakukan hal-hal di atas berulang kali saat melakukan training per-epoch. Cache dari data dapat disimpan di memori maupun di penyimpanan lokal. Caranya yaitu memanggil ```.cache()``` pada data 

In [11]:
tf_dataset_2= tf_dataset_2.cache()

Proses yang di-cache adalah proses-proses yang dipanggil sebelum ```.cache()```

**Prefetch**

Prefetching artinya membuat semua proses terjadi secara tumpang tindih atau parallel, mulai dari pemrosesan, load data per-batch, hingga training pada data

In [13]:
tf_dataset_2= tf_dataset_2.prefetch(tf.data.AUTOTUNE)

Benchmarking